In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
!gsutil cp -r gs://gresearch/goemotions/data/full_dataset/ .

Copying gs://gresearch/goemotions/data/full_dataset/goemotions_1.csv...
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_2.csv...
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_3.csv...
\ [3 files][ 40.8 MiB/ 40.8 MiB]                                                
Operation completed over 3 objects/40.8 MiB.                                     


In [4]:
!pip install -U transformers accelerate datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


In [5]:
import pandas as pd

df1 = pd.read_csv("/content/full_dataset/goemotions_1.csv")
df2 = pd.read_csv("/content/full_dataset/goemotions_2.csv")
df3 = pd.read_csv("/content/full_dataset/goemotions_3.csv")

In [6]:
frames = [df1,df2,df3]
df = pd.concat(frames)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 211225 entries, 0 to 71224
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   text                  211225 non-null  object 
 1   id                    211225 non-null  object 
 2   author                211225 non-null  object 
 3   subreddit             211225 non-null  object 
 4   link_id               211225 non-null  object 
 5   parent_id             211225 non-null  object 
 6   created_utc           211225 non-null  float64
 7   rater_id              211225 non-null  int64  
 8   example_very_unclear  211225 non-null  bool   
 9   admiration            211225 non-null  int64  
 10  amusement             211225 non-null  int64  
 11  anger                 211225 non-null  int64  
 12  annoyance             211225 non-null  int64  
 13  approval              211225 non-null  int64  
 14  caring                211225 non-null  int64  
 15  confus

In [7]:
df = df.head(25000).copy()
df = df[df["example_very_unclear"] == 0]
columns = ["author", "link_id", "parent_id", "created_utc", "rater_id", "subreddit", "example_very_unclear", "id"]
df.drop(columns, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24630 entries, 0 to 24999
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            24630 non-null  object
 1   admiration      24630 non-null  int64 
 2   amusement       24630 non-null  int64 
 3   anger           24630 non-null  int64 
 4   annoyance       24630 non-null  int64 
 5   approval        24630 non-null  int64 
 6   caring          24630 non-null  int64 
 7   confusion       24630 non-null  int64 
 8   curiosity       24630 non-null  int64 
 9   desire          24630 non-null  int64 
 10  disappointment  24630 non-null  int64 
 11  disapproval     24630 non-null  int64 
 12  disgust         24630 non-null  int64 
 13  embarrassment   24630 non-null  int64 
 14  excitement      24630 non-null  int64 
 15  fear            24630 non-null  int64 
 16  gratitude       24630 non-null  int64 
 17  grief           24630 non-null  int64 
 18  joy        

In [8]:
LABELS = ['admiration','amusement','anger','annoyance','approval','caring',
    'confusion','curiosity','desire','disappointment','disapproval','disgust',
    'embarrassment','excitement','fear','gratitude','grief','joy','love',
    'nervousness','optimism','pride','realization','relief','remorse',
    'sadness','surprise','neutral',
]


In [9]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# First split: train vs (val+test)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)

# Second split: validation vs test
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)

print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

# convert to hugginface dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

Train size: 17241
Validation size: 3694
Test size: 3695


In [10]:
# Tokenizer converts raw text into IDs

from transformers import AutoTokenizer

model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/17241 [00:00<?, ? examples/s]

Map:   0%|          | 0/3694 [00:00<?, ? examples/s]

Map:   0%|          | 0/3695 [00:00<?, ? examples/s]

In [11]:
import torch

label_cols = df.columns.drop("text").tolist()  # all 28 emotions
num_labels = len(label_cols)

# We collect them into a single column called "labels", because Hugging Face expects that.
def format_labels(example):
    example["labels"] = torch.tensor([float(example[col]) for col in label_cols], dtype=torch.float)
    return example

train_dataset = train_dataset.map(format_labels)
val_dataset = val_dataset.map(format_labels)
test_dataset = test_dataset.map(format_labels)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/17241 [00:00<?, ? examples/s]

Map:   0%|          | 0/3694 [00:00<?, ? examples/s]

Map:   0%|          | 0/3695 [00:00<?, ? examples/s]

In [12]:
batch = train_dataset[0]
print(batch["labels"])
print(batch["labels"].dtype)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
torch.float32


In [13]:
from transformers import AutoModelForSequenceClassification
# Loads a pretrained transformer
# Uses sigmoid activation (multi-label)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16)
batch = next(iter(train_loader))

print(batch['labels'].shape)  # should be (16, 28)
print(batch['labels'].dtype)  # should be float, casted to int later


torch.Size([16, 28])
torch.float32


In [18]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
accuracy_metric = evaluate.load("accuracy")


# logits = raw model outputs
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    predictions = (probs > 0.5).astype(np.int32)

    # Ensure labels are np.int32 type
    labels = labels.astype(np.int32)

    # Flatten predictions and labels
    predictions = predictions.flatten()
    labels = labels.flatten()

    f1 = f1_metric.compute(predictions=predictions, references=labels, average="micro")
    precision = precision_metric.compute(predictions=predictions, references=labels, average="micro")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="micro")
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)

    return {"f1": f1["f1"], "precision": precision["precision"], "recall": recall["recall"], "accuracy": accuracy["accuracy"]}

training_args = TrainingArguments(
    output_dir="./results",            # where to save the model
    eval_strategy="epoch",             # run evaluation at the end of each epoch
    save_strategy="epoch",             # save checkpoint each epoch
    learning_rate=2e-5,                # standard LR for fine-tuning
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,
    load_best_model_at_end=True,       # reload best model when finished
    metric_for_best_model="f1",        # depends on your compute_metrics
    greater_is_better=True,            # if higher metric is better
    report_to=[],
    fp16=True
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy
1,0.084100,0.132491,0.957866,0.957866,0.957866,0.957866
2,0.097300,0.133875,0.955546,0.955546,0.955546,0.955546
3,0.088600,0.134000,0.956464,0.956464,0.956464,0.956464


TrainOutput(global_step=3234, training_loss=0.08903074876423153, metrics={'train_runtime': 560.7996, 'train_samples_per_second': 92.231, 'train_steps_per_second': 5.767, 'total_flos': 3403017506036736.0, 'train_loss': 0.08903074876423153, 'epoch': 3.0})

In [19]:
# Save the trained model
trainer.evaluate(test_dataset)


{'eval_loss': 0.12954728305339813,
 'eval_f1': 0.9588246665377924,
 'eval_precision': 0.9588246665377924,
 'eval_recall': 0.9588246665377924,
 'eval_accuracy': 0.9588246665377924,
 'eval_runtime': 7.7206,
 'eval_samples_per_second': 478.592,
 'eval_steps_per_second': 29.92,
 'epoch': 3.0}

In [ ]:
trainer.save_model("./trained_model")